changelog:

* 4/16/19: added newagg
* 4/17/19: reformatted the ordering of how code runs, and variable names. added aggregation #3.
* 4/19/19: changed the standardizing so that log(x+1) is now applied prior to standardization. also removed ordinal variables from standardizing algorithm and concat them in later with median 0 and iqr 1 so standardize value is either 0 or 1. values from ordinal are not log transformed.


06/14/19:
# Overview of this notebook:

1. convert to two class (c-/abshort &c+/ablong)
2. split the train and test set
3. median standardize
4. aggregate
5. impute

i do 1-3 BEFORE I AGGREGATE and median standardize, as some information may be leaking from the train/test set as i had it previously.

In [1]:
!pip install scikit-learn 1.3.0

ERROR: Could not find a version that satisfies the requirement 1.3.0 (from versions: none)
ERROR: No matching distribution found for 1.3.0


In [2]:
pip list

Package                       Version
----------------------------- ---------
aiohttp                       3.8.4
aiosignal                     1.3.1
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.2.1
async-timeout                 4.0.2
asyncio                       3.4.3
attrs                         23.1.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.5
beautifulsoup4                4.12.2
bleach                        6.0.0
bs4                           0.0.1
caugetch                      0.0.1
certifi                       2023.5.7
cffi                          1.15.1
charset-normalizer            3.2.0
clipboard                     0.0.4
colorama                      0.4.6
contourpy                     1.1.0
cycler                        0.11.0
DateTime                      5.2
debugpy                       1.5.1
decorator                     5.1.1
defusedxml                    0.7.1
executing      

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob
import sklearn

from sklearn.model_selection import train_test_split
# from joblib import Memory
# memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

time: 0 ns (started: 2023-07-27 12:28:39 +08:00)


In [4]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path, categorical

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v #pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0)
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['stay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 3.84 s (started: 2023-07-27 12:28:39 +08:00)


In [5]:
len(patients)

20464

time: 0 ns (started: 2023-07-27 12:28:43 +08:00)


In [6]:

def save_df(df, df_name='default', add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    
    save_path= str(repository_path)+'/data/cleaned_merged_agg/'
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_cleaned_merged_agg.csv' %(date, df_name)))

time: 16 ms (started: 2023-07-27 12:28:43 +08:00)


In [7]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df


time: 0 ns (started: 2023-07-27 12:28:43 +08:00)


In [8]:
#importing cleaned_merged big_df
allFiles = glob.glob(str(repository_path)+ '/data/cleaned_merged/%s/'%(folder) + "{}_*.csv".format(date))

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('_prepped.csv')[0]) #making an list of all my dataframes in order they appear in file

i=0
for name in df_list:
    big_df = pd.read_csv(allFiles[i],  index_col=0)

time: 7.23 s (started: 2023-07-27 12:28:43 +08:00)


In [9]:
#quick housekeeping addition to accomidate older generated data
if len(big_df.loc[big_df['label']=="pao2/fio2",'label'])>1:
    big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2ratio"

time: 437 ms (started: 2023-07-27 12:28:50 +08:00)


In [10]:
big_df.head()

,stay_id,t_0,value,label,uom,delta,source,subject_id
22699,30000484,2136-01-14 21:00:00,0.0,cancer_elix,y/n,0 days 00:00:00,cancer_elix,18421337
22479,30000484,2136-01-14 21:00:00,0.0,epinephrine,y/n,0 days 00:00:00,epinephrine,18421337
50943,30000484,2136-01-14 21:00:00,M,gender,NaN,0 days 00:00:00,pt_info,18421337
12573,30000484,2136-01-14 21:00:00,163.0,height,cm,0 days 00:00:00,height,18421337
23219,30000484,2136-01-14 21:00:00,0.0,norepinephrine,y/n,0 days 00:00:00,norepinephrine,18421337


time: 15 ms (started: 2023-07-27 12:28:51 +08:00)


In [11]:
#overview of all variables and formats
big_df.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
any_vasoactives,26471,2,0.0,17954
vent_recieved,22444,2,Mech,12330
vasopressin,26471,2,0.0,26470
rrt,26471,2,0.0,24251
cancer_elix,26471,2,0.0,24097
phenylephrine,26471,2,0.0,22828
o2_flow,26471,2,0.0,25544
dobutamine,26471,2,0.0,26102
dopamine,26471,2,0.0,25917


time: 1.33 s (started: 2023-07-27 12:28:51 +08:00)


## initial data prep
* convert to two class

In [12]:
#convert to two class
final_pt_df2['final_bin'].unique()
two_classes=['C_neg/A_partial','C_pos/A_full']
two_class_icu=final_pt_df2.loc[final_pt_df2.loc[:,"final_bin"].isin(two_classes),['stay_id','subject_id','final_bin']]


time: 15 ms (started: 2023-07-27 12:28:52 +08:00)


In [13]:
big_df=big_df.loc[big_df['stay_id'].isin(list(two_class_icu['stay_id'])),:].copy()

time: 484 ms (started: 2023-07-27 12:28:52 +08:00)


In [14]:
len(big_df)
big_df['stay_id'].nunique() 

16118

time: 47 ms (started: 2023-07-27 12:28:53 +08:00)


### roundabout way of sampling train/test set so that each subject is only in either train or test:
* To ensure a single patient did not end up with samples in both training and testing sets, individual patients (subject_id's) are kept together when performing the stratified train and test split

In [15]:
# label each subject_id with the max of the two classes. 
two_class_pt=two_class_icu.copy()
two_class_pt['final_bin']=pd.factorize(two_class_pt['final_bin'])[0]
two_class_maxsub=two_class_pt.loc[two_class_pt.groupby('subject_id')['final_bin'].idxmax(),:]

time: 828 ms (started: 2023-07-27 12:28:53 +08:00)


In [16]:
# 70/30 train/test set split with 12345=seed, splitting on max final bin of each SUBJECT_ID
train, test = train_test_split(two_class_maxsub, test_size=0.3, random_state=12345, 
                               stratify=two_class_maxsub['final_bin'])
# generate list of each SUBJECT_ID in each split
train_subject=list(train['subject_id'])
test_subject=list(test['subject_id'])

#filtering big_df on train subjects and test subjects to get my train/test splits.
big_df_train= big_df.loc[big_df.loc[:,'subject_id'].isin(train_subject),:].copy()
big_df_test= big_df.loc[big_df.loc[:,'subject_id'].isin(test_subject),:].copy()
del big_df

time: 516 ms (started: 2023-07-27 12:28:54 +08:00)


## calc median/iqr of 'healthiest patients' for standardization
take all non-categorical variables for HEALTHY PATIENTS and calculate the median and IQR for them. then will use this to make z scores via:

$$Z=\frac{(X-\widetilde{X}_{-/short})}{(IQR_{-/short})}$$ where $\widetilde{X}_{-/short}$ is the median value of the patients with negative SSC and short duration EAT.


In [17]:
big_df_train[big_df_train['label']=='vent_recieved']['stay_id'].nunique()

10921

time: 156 ms (started: 2023-07-27 12:28:54 +08:00)


In [18]:
big_df_train['stay_id'].nunique()

11290

time: 47 ms (started: 2023-07-27 12:28:54 +08:00)


In [19]:
from parameters import continuous, onetime


time: 0 ns (started: 2023-07-27 12:28:54 +08:00)


In [20]:
def median_label_fxn(big_df):
    """ grabs the median and IQR values for all continuous variables based upon the values present in
    the last 24 hours of the c_neg/A_partial patients (AKA the healthiest patients at the healthiest timepoints) """
    
    global final_pt_df2
    healthy_pt=list(final_pt_df2[final_pt_df2['final_bin']=="C_neg/A_partial"]['stay_id'])

    #filter to only healthy patients filter
    ##splitting big_df, making a copy and restricting it to all values that will be usd in standardizing
    healthy_df=big_df[big_df['label'].isin(continuous+onetime)].copy()
    healthy_df['value']= pd.to_numeric(healthy_df['value']) #converting to numeric 
    healthy_df=healthy_df[healthy_df['stay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt
    
    #finding the last 24 hours of each healthy patient.
    healthy_pt_end=pd.DataFrame(healthy_df.groupby("stay_id")['delta'].max())
    healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
    healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()
    healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="stay_id", right_on="stay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
    
    #calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
    median_label=pd.DataFrame((healthy_df.groupby("label")['value'].median())).reset_index()
    median_label=median_label.rename(columns={'value':"median"})
    iqr_label=pd.DataFrame((healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25))).reset_index()
    iqr_label=iqr_label.rename(columns={'value':"iqr"})
    median_label=pd.merge(median_label,iqr_label)  #final median df
    return(median_label)

def standardization_fxn(big_df):
    global continuous, onetime, vaso_active, ordinal, categorical
    
    median_label=median_label_fxn(big_df)
    
    ### dataformatting: convert all dtypes to a numeric type that pereserves nan. 
    #splitting categorical, ordinal and continuous
    big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy() 

    #continuous and ordinal variables
    big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous+onetime),:].copy() 
    big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 
    
    ### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 
    big_noCat=pd.merge(big_noCat, median_label, how="left") 
    #loging values
    big_noCat['median']= np.log(big_noCat['median']+1.0)
    big_noCat['iqr']= np.log(big_noCat['iqr']+1.0)
    big_noCat['raw_value']=big_noCat['value']
    big_noCat['value']=np.log(big_noCat['value']+1.0)
    
    big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0) #standardize is log standardized  

    return(big_noCat, big_categorical)

time: 15 ms (started: 2023-07-27 12:28:54 +08:00)


In [21]:
noCat_df_train, cat_df_train = standardization_fxn(big_df_train)
noCat_df_test, cat_df_test = standardization_fxn(big_df_test)

time: 46.4 s (started: 2023-07-27 12:28:55 +08:00)


In [22]:
cat_df_train.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
any_vasoactives,11290,2,0.0,7972
cancer_elix,11290,2,0.0,10372
dobutamine,11290,2,0.0,11153
dopamine,11290,2,0.0,11131
epinephrine,11290,2,0.0,10914
gender,11290,2,M,6274
norepinephrine,11290,2,0.0,9312
o2_flow,11290,2,0.0,10951
phenylephrine,11290,2,0.0,9720


time: 63 ms (started: 2023-07-27 12:29:46 +08:00)


In [23]:
noCat_df_train.groupby('label')['value'].describe()

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
bicarbonate,21521.0,3.141861,0.225969,1.609438,3.044522,3.178054,3.295837,3.912023
bilirubin,6770.0,0.961129,0.776138,0.095310,0.405465,0.693147,1.308333,4.282206
bun,21468.0,3.214693,0.701587,0.693147,2.708050,3.178054,3.713572,5.446737
calcium,12629.0,0.751306,0.055361,0.292670,0.722706,0.751416,0.779325,1.420696
chloride,24736.0,4.652376,0.069579,1.481605,4.615121,4.653960,4.691348,5.017280
creatinine,21527.0,0.879092,0.435623,0.095310,0.587787,0.741937,1.098612,3.178054
diasbp,278456.0,4.101070,0.228413,2.772589,3.951244,4.094345,4.248495,5.509388
glucose,68613.0,4.921361,0.371327,0.113329,4.682131,4.875197,5.123964,14.059322
heartrate,282929.0,4.464840,0.208879,3.433987,4.330733,4.465908,4.605170,5.541264


time: 344 ms (started: 2023-07-27 12:29:50 +08:00)


In [24]:
cat_df_train

,stay_id,t_0,value,label,uom,delta,source,subject_id
16956,30002654,2154-10-17 20:00:00,0.0,cancer_elix,y/n,0 days 00:00:00,cancer_elix,15978672
34747,30002654,2154-10-17 20:00:00,M,gender,NaN,0 days 00:00:00,pt_info,15978672
17415,30002654,2154-10-17 20:00:00,0.0,phenylephrine,y/n,0 days 00:00:00,phenylephrine,15978672
61218,30002654,2154-10-17 20:00:00,white/nonhispanic,race,NaN,0 days 00:00:00,pt_info,15978672
16906,30002654,2154-10-17 20:00:00,0.0,rrt,y/n,0 days 00:00:00,rrt,15978672
...,...,...,...,...,...,...,...,...
5332,39999552,2186-07-17 14:00:00,0.0,rrt,y/n,0 days 00:00:00,rrt,11256534
3363,39999552,2186-07-17 14:00:00,0.0,vasopressin,y/n,0 days 00:00:00,vasopressin,11256534
1302,39999552,2186-07-17 14:00:00,1.0,phenylephrine,y/n,0 days 02:46:00,phenylephrine,11256534
4106,39999552,2186-07-17 14:00:00,1.0,any_vasoactives,y/n,0 days 07:05:00,all_vaso,11256534


time: 16 ms (started: 2023-07-27 12:29:54 +08:00)


In [24]:
#should be same number of col as len(categorical)
cat_df_train[cat_df_train['stay_id']==30002654].groupby('label')['value'].value_counts()#.head(10)

label            value 
any_vasoactives  1.0       1
bands            >10       1
cancer_elix      0.0       1
dobutamine       0.0       1
dopamine         0.0       1
epinephrine      0.0       1
gender           M         1
norepinephrine   1.0       1
o2_flow          0.0       1
pco2             <50       1
phenylephrine    0.0       1
rrt              0.0       1
vasopressin      0.0       1
vent_recieved    Oxygen    1
Name: count, dtype: int64

time: 0 ns (started: 2023-07-27 08:33:04 +08:00)


In [25]:
noCat_df_train[noCat_df_train['label']=='lactate']

,stay_id,t_0,value,label,uom,delta,source,subject_id,median,iqr,raw_value,standardize
111,30002654,2154-10-17 20:00:00,1.252763,lactate,mmol/L,0 days 18:15:00,labs,15978672,1.098612,1.029619,2.5,0.149716
596,30005366,2202-12-28 15:00:00,0.530628,lactate,mmol/L,0 days 03:30:00,labs,18855147,1.098612,1.029619,0.7,-0.551645
630,30005366,2202-12-28 15:00:00,0.530628,lactate,mmol/L,0 days 07:28:00,labs,18855147,1.098612,1.029619,0.7,-0.551645
1373,30006983,2159-10-12 05:00:00,1.064711,lactate,mmol/L,0 days 03:14:00,labs,18756985,1.098612,1.029619,1.9,-0.032926
1408,30006983,2159-10-12 05:00:00,1.435085,lactate,mmol/L,0 days 12:04:00,labs,18756985,1.098612,1.029619,3.2,0.326793
...,...,...,...,...,...,...,...,...,...,...,...,...
2250931,39999230,2147-08-31 22:00:00,1.774952,lactate,mmol/L,0 days 04:09:00,labs,13651601,1.098612,1.029619,4.9,0.656884
2250959,39999230,2147-08-31 22:00:00,1.335001,lactate,mmol/L,0 days 06:14:00,labs,13651601,1.098612,1.029619,2.8,0.229589
2250985,39999230,2147-08-31 22:00:00,1.193922,lactate,mmol/L,0 days 08:25:00,labs,13651601,1.098612,1.029619,2.3,0.092568
2251034,39999230,2147-08-31 22:00:00,1.308333,lactate,mmol/L,0 days 14:32:00,labs,13651601,1.098612,1.029619,2.7,0.203687


time: 125 ms (started: 2023-07-27 12:30:01 +08:00)


# aggregation1:
### clincally guided min/max

In [26]:
from parameters import low_value, hi_value, both_value

#important_onetime=['yearsold','weight'] #pco2 and bands are now in categorical 12-12-19


time: 16 ms (started: 2023-07-27 12:30:04 +08:00)


In [27]:
noCat_df_train['label'].unique()#noCat_df_train['label']=='any_vasoactives'

array(['sum_elix', 'weight', 'yearsold', 'pao2fio2ratio', 'mingcs',
       'spo2', 'heartrate', 'resprate', 'diasbp', 'meanartpress', 'sysbp',
       'temperature', 'bicarbonate', 'bun', 'chloride', 'creatinine',
       'glucose', 'hemoglobin', 'inr', 'platelet', 'potassium', 'ptt',
       'sodium', 'wbc', 'ph', 'bilirubin', 'lactate', 'calcium'],
      dtype=object)

time: 110 ms (started: 2023-07-27 12:30:06 +08:00)


In [28]:
def clin_agg(big_noCat, big_cat, big_df, values="standardize"):
    """
    clincally guided aggregations.
    
    values= choose here if want to use standardization or raw values.
    
    note: getting two minor errors, could use some cleaning up at later date.
    """
    global hi_value, low_value, both_value, important_onetime
    #max aggregation for selected variables
    big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
    table = pd.pivot_table(big_max, values=values, columns='label', index=['stay_id'],aggfunc=max, dropna=False)
    
    #min aggregation for selected variables
    big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
    table2 = pd.pivot_table(big_min, values=values, columns='label', index=['stay_id'],aggfunc=min, dropna=False)
    
    #max&min aggregation for selected variables
    big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
    table3 = pd.pivot_table(big_both, values=values, columns='label', index=['stay_id'],aggfunc=[max,min], dropna=False) #different level
    table4 = pd.merge(table3['max'].reset_index(), table3['min'].reset_index(), left_on='stay_id', right_on='stay_id', how = 'left') #change to 1

    #first left join all different continuous aggregations together. 
    worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
    worst_df=pd.merge(worst_df, table4, left_on='stay_id', right_on='stay_id',how='left')
    
    ### formatting categorical to wide format to match the tables/worst_df
    big_cat= big_cat.pivot(
    index='stay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per icustay per time. 
    
    ## merging the categorical and aggregated dataframes together. 
    worst_df=pd.merge(worst_df, big_cat, left_on='stay_id', right_on='stay_id',how='left') #using max/min aggregates 

#     #adding important one_time values to final aggregated
#     agg_remaining= big_df.loc[big_df.loc[:,'label'].isin(important_onetime),:]
#     agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
#     agg_table2.columns = agg_table2.columns.get_level_values(1)
#     agg_table2=agg_table2.reset_index()
#     agg_table2.head()#.rename(columns={})

#     worst_df=pd.merge(worst_df, agg_table2, how='left')
    return(worst_df)


time: 0 ns (started: 2023-07-27 12:30:14 +08:00)


In [36]:
big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both, values=values, columns='label', index=['stay_id'],aggfunc=[max,min], dropna=False)

time: 188 ms (started: 2023-07-27 08:43:55 +08:00)


In [37]:
table3

max                           min                    
label      calcium    sodium       wbc   calcium    sodium       wbc
stay_id                                                             
30002654       NaN  0.010973  0.203706       NaN  0.007342  0.203706
30004144       NaN -0.007448  0.078416       NaN -0.011213 -0.026804
30005085 -0.228703 -0.007448 -0.015124 -0.228703 -0.011213 -0.015124
30005366       NaN  0.003684  0.035756       NaN  0.003684  0.035756
30005474  0.045093  0.021717  0.096840  0.045093  0.021717  0.096840
...            ...       ...       ...       ...       ...       ...
39996123 -0.275106  0.000000  0.349381 -0.275106 -0.007448  0.222380
39996867       NaN -0.003710  0.166494       NaN -0.003710  0.166494
39999172       NaN  0.028757 -0.291758       NaN  0.028757 -0.291758
39999230  0.089975  0.007342 -0.232437  0.089975  0.000000 -0.320812
39999552  0.398394  0.018161 -0.059492 -0.368592 -0.007448 -0.127895

[11032 rows x 6 columns]

time: 16 ms (started: 2023-07-27 08:43:56 +08:00)


In [51]:
table4 = pd.merge(table3['max'].reset_index(), table3['min'].reset_index(), left_on='stay_id', right_on='stay_id', how = 'left')

time: 16 ms (started: 2023-07-27 09:01:26 +08:00)


In [52]:
table4

label,stay_id,calcium_x,sodium_x,wbc_x,calcium_y,sodium_y,wbc_y
0,30002654,NaN,0.010973,0.203706,NaN,0.007342,0.203706
1,30004144,NaN,-0.007448,0.078416,NaN,-0.011213,-0.026804
2,30005085,-0.228703,-0.007448,-0.015124,-0.228703,-0.011213,-0.015124
3,30005366,NaN,0.003684,0.035756,NaN,0.003684,0.035756
4,30005474,0.045093,0.021717,0.096840,0.045093,0.021717,0.096840
...,...,...,...,...,...,...,...
11027,39996123,-0.275106,0.000000,0.349381,-0.275106,-0.007448,0.222380
11028,39996867,NaN,-0.003710,0.166494,NaN,-0.003710,0.166494
11029,39999172,NaN,0.028757,-0.291758,NaN,0.028757,-0.291758
11030,39999230,0.089975,0.007342,-0.232437,0.089975,0.000000,-0.320812


time: 15 ms (started: 2023-07-27 09:01:27 +08:00)


In [29]:
worst_df_train=clin_agg(noCat_df_train, cat_df_train,big_df_train, values="standardize")

time: 984 ms (started: 2023-07-27 12:31:19 +08:00)


In [30]:
worst_df_test=clin_agg(noCat_df_test, cat_df_test, big_df_test, values="standardize")

time: 422 ms (started: 2023-07-27 12:31:24 +08:00)


In [31]:
worst_df_train.head()

label,stay_id,bilirubin,bun,chloride,creatinine,glucose,heartrate,inr,lactate,potassium,...,epinephrine,gender,norepinephrine,o2_flow,pco2,phenylephrine,race,rrt,vasopressin,vent_recieved
0,30002654,-0.26186,0.025592,0.044936,0.435041,-0.015390,0.030582,0.485147,0.149716,0.033036,...,0.0,M,1.0,0.0,<50,0.0,white/nonhispanic,0.0,0.0,Oxygen
1,30004144,NaN,-0.092778,-0.013067,-0.207768,0.000000,0.078472,0.000000,NaN,0.065443,...,0.0,M,0.0,0.0,<50,1.0,white/nonhispanic,0.0,0.0,Mech
2,30005085,NaN,-0.131194,-0.008669,-0.134895,0.024799,0.049416,-0.224364,NaN,0.097243,...,0.0,M,0.0,0.0,absent,0.0,white/nonhispanic,0.0,0.0,Oxygen
3,30005366,-0.32466,0.000000,-0.031093,1.603348,-0.036101,0.013965,2.636968,-0.551645,0.189228,...,0.0,F,0.0,0.0,>50,0.0,white/nonhispanic,1.0,0.0,Mech
4,30005474,NaN,0.101360,0.016858,0.179977,0.080308,-0.037933,-0.109632,NaN,0.189228,...,0.0,M,0.0,0.0,absent,0.0,white/nonhispanic,0.0,0.0,Oxygen


time: 15 ms (started: 2023-07-27 12:31:26 +08:00)


In [35]:
worst_df_train['pao2fio2ratio'].describe()

count    11290.000000
mean         0.107286
std          0.154993
min         -0.718686
25%         -0.022491
50%          0.216303
75%          0.216303
max          0.216303
Name: pao2fio2ratio, dtype: float64

time: 0 ns (started: 2023-07-27 13:53:25 +08:00)


In [36]:
worst_df_train['vasopressin'].value_counts()

vasopressin
0.0    11289
1.0        1
Name: count, dtype: int64

time: 0 ns (started: 2023-07-27 13:53:27 +08:00)


In [37]:
worst_df_train['bands'].value_counts()

bands
absent    10470
<10         588
>10         232
Name: count, dtype: int64

time: 0 ns (started: 2023-07-27 13:53:30 +08:00)


In [38]:
worst_df_train['pco2'].value_counts()

pco2
<50       4956
absent    4710
>50       1624
Name: count, dtype: int64

time: 0 ns (started: 2023-07-27 13:53:33 +08:00)


In [39]:
save_df(worst_df_train, 'train')
save_df(worst_df_test, 'test')

time: 719 ms (started: 2023-07-27 13:53:35 +08:00)


In [70]:
del worst_df_train, worst_df_test

time: 0 ns (started: 2023-07-27 09:12:13 +08:00)
